# Sports Tracking Data Visualization in VR

Analyze player movement and game events in 3D space.

In [ ]:
# !pip install numpy pandas requests immersivepoints statsbombpy

In [ ]:
import numpy as np
import pandas as pd
import immersivepoints as ip

print("Sports Analytics VR Ready!")

## Option 1: Load StatsBomb Open Data (Soccer Events)

In [ ]:
try:
    from statsbombpy import sb
    
    print("Loading StatsBomb data...")
    
    # Get available competitions
    competitions = sb.competitions()
    print(f"\nAvailable competitions: {len(competitions)}")
    print(competitions[['competition_name', 'season_name']].head())
    
    # Example: World Cup 2018
    matches = sb.matches(competition_id=43, season_id=3)  # World Cup 2018
    print(f"\nFound {len(matches)} matches")
    
    # Load events from a match
    match_id = matches.iloc[0]['match_id']
    events = sb.events(match_id=match_id)
    
    print(f"Loaded {len(events)} events from match {match_id}")
    print(f"Teams: {matches.iloc[0]['home_team']} vs {matches.iloc[0]['away_team']}")
    
except ImportError:
    print("StatsBomb library not installed.")
    print("Install with: pip install statsbombpy")
    print("Using synthetic data instead...")
    events = None

## Option 2: Generate Synthetic Player Tracking Data

In [ ]:
def generate_match_tracking(n_players=22, duration_minutes=90, fps=1):
    """
    Generate synthetic player tracking data for a soccer match.
    Field: 105m × 68m
    """
    field_length = 105  # meters
    field_width = 68
    
    timestamps = np.arange(0, duration_minutes * 60, 1.0/fps)  # seconds
    n_frames = len(timestamps)
    
    all_positions = []
    
    for player_id in range(n_players):
        team = 0 if player_id < 11 else 1
        
        # Starting position based on formation
        if team == 0:  # Home team attacks right
            start_x = field_length * 0.3
        else:  # Away team attacks left
            start_x = field_length * 0.7
        
        start_y = field_width * (0.2 + 0.6 * (player_id % 11) / 10)
        
        # Random walk with bounds
        x = np.cumsum(np.random.randn(n_frames) * 0.5) + start_x
        y = np.cumsum(np.random.randn(n_frames) * 0.3) + start_y
        
        # Keep on field
        x = np.clip(x, 0, field_length)
        y = np.clip(y, 0, field_width)
        
        # Add time as third dimension
        z = timestamps / 60  # minutes
        
        # Hue by team
        hue = np.full(n_frames, 0.0 if team == 0 else 0.6)  # Red vs Blue
        
        positions = np.column_stack([x, y, z, hue])
        all_positions.append(positions)
    
    # Combine all players
    tracking_data = np.vstack(all_positions)
    
    return tracking_data

print("Generating synthetic match data...")
tracking = generate_match_tracking(n_players=22, duration_minutes=90, fps=0.5)
print(f"Generated {len(tracking):,} position records")
print(f"({22} players × {90*0.5:.0f} frames)")

## Process StatsBomb Event Data (if loaded)

In [ ]:
if events is not None:
    # Extract location data
    event_positions = []
    
    for idx, event in events.iterrows():
        if 'location' in event and event['location'] is not None:
            x, y = event['location']
            
            # Time in minutes
            time = event['minute']
            
            # Color by team
            team_id = event['team_id']
            hue = 0.0 if team_id == events.iloc[0]['team_id'] else 0.6
            
            event_positions.append([x, y, time, hue])
    
    tracking = np.array(event_positions, dtype=np.float32)
    print(f"\nProcessed {len(tracking)} events with locations")
else:
    print("Using synthetic tracking data")

## Prepare for VR

Scale field to comfortable VR size.

In [ ]:
# Extract coordinates
x = tracking[:, 0]  # Field X (length)
y = tracking[:, 1]  # Field Y (width)
z = tracking[:, 2]  # Time or importance
hue = tracking[:, 3]  # Team color

# Center field
x_centered = x - x.mean()
y_centered = y - y.mean()

# Scale for VR (1 meter = 0.1 VR units)
scale = 0.1
x_vr = x_centered * scale
y_vr = y_centered * scale
z_vr = z * scale * 2  # Exaggerate vertical for visibility

# Create point cloud
points = np.column_stack([x_vr, y_vr, z_vr, hue]).astype(np.float32)

print(f"\nVR Point Cloud:")
print(f"  Points: {len(points):,}")
print(f"  Field size: {x.max()-x.min():.0f}m × {y.max()-y.min():.0f}m")
print(f"  Duration: {z.max():.0f} minutes")
print(f"  VR dimensions: X=[{x_vr.min():.1f}, {x_vr.max():.1f}] "
      f"Y=[{y_vr.min():.1f}, {y_vr.max():.1f}] "
      f"Z=[{z_vr.min():.1f}, {z_vr.max():.1f}]")

## Visualize in Jupyter

In [ ]:
ip.renderPoints(points, point_size=0.05, background_color=0x228B22, show_axes=True)

## Create Heat Map View

Flatten time to see position density.

In [ ]:
# Create 2D heatmap (flatten time dimension)
points_2d = np.column_stack([x_vr, y_vr, np.zeros_like(z_vr), hue]).astype(np.float32)

print(f"Created 2D heatmap with {len(points_2d):,} positions")

# Visualize
ip.renderPoints(points_2d, point_size=0.04, background_color=0x228B22)

## Generate VR Links

In [ ]:
print("3D Trajectory View (Time as Z-axis):")
ip.showVR(points, point_size=0.05)

print("\n2D Heatmap View:")
ip.showVR(points_2d, point_size=0.04)

## Analyze Specific Patterns

Extract interesting tactical moments.

In [ ]:
# Find high-intensity periods (many events)
time_bins = np.linspace(0, z.max(), 20)
event_counts, _ = np.histogram(z, bins=time_bins)

most_active = np.argmax(event_counts)
print(f"Most active period: {time_bins[most_active]:.0f}-{time_bins[most_active+1]:.0f} minutes")
print(f"Events in this period: {event_counts[most_active]}")

# Field coverage by team
home_mask = (hue < 0.3)
away_mask = (hue > 0.3)

print(f"\nHome team avg position: X={x[home_mask].mean():.1f}m, Y={y[home_mask].mean():.1f}m")
print(f"Away team avg position: X={x[away_mask].mean():.1f}m, Y={y[away_mask].mean():.1f}m")

## Save for Upload

In [ ]:
# Save 3D trajectory
output_3d = "match_trajectory_3d.xyzi"
points.astype(np.float32).byteswap().tofile(output_3d)
print(f"Saved 3D view to {output_3d}")

# Save 2D heatmap
output_2d = "match_heatmap_2d.xyzi"
points_2d.astype(np.float32).byteswap().tofile(output_2d)
print(f"Saved 2D heatmap to {output_2d}")

print(f"\nUpload at: https://immersivepoints.com/upload.html")

## What to Explore in VR

**In 3D Trajectory View:**
- Walk through time to see player movement evolution
- See where teams spent most time (vertical density)
- Identify pressing patterns and defensive shape

**In 2D Heatmap View:**
- See territorial dominance (red vs blue density)
- Find attacking zones and defensive weaknesses
- Compare individual player heat maps

**Try These Analyses:**
- Filter to specific time periods (e.g., first 15 minutes)
- Extract single player trajectories
- Color by speed or distance from ball
- Add event markers (shots, passes, tackles) as larger points